In [1]:
import pandas as pd
import matplotlib as pt
import alpaca_trade_api as tradeapi
import requests
import yfinance as yf
from datetime import datetime, timedelta

In [3]:
# Alpaca API config stuff

APCA_API_BASE_URL = 'https://paper-api.alpaca.markets'
APCA_MARKET_DATA = 'https://data.alpaca.markets/v1'
APCA_API_KEY = 'PKDQTHU4Z3R8ZTXFOGXN'
APCA_SECRET_KEY = 'FBE3WStDzqd6WFATRUVQbDLRlht0E6p7kO6gLOdB'

api = tradeapi.REST(APCA_API_KEY, APCA_SECRET_KEY, APCA_API_BASE_URL)

In [4]:
# IEX API config stuff

IEX_CLOUD_API_KEY = 'Tpk_12e6eee6ed7d4026a0a87dee063b86bd'
IEX_CLOUD_SECRET_KEY = 'Tsk_620c4eafd98f479f990e532c3a0b2cb0'
SANDBOX_BASE_URL = 'https://sandbox.iexapis.com/stable/'